# 0. IMPORTANT--Specify classifier to be trained and dataset

In [1]:
model_name = 'lgr'  # options include: {'Decision Tree': 'dt', 'Gaussian Naive Bayes':'gnb',\
                           #                  'Logistic Regression': 'lgr', \
                           #                  'Gradient_Boosted_Trees': 'gbt'} 
data_file = '/home/mackenzie/git_repositories/delayedimpact/data/simData_oom100.csv'  # ...oom10, ...oom50, ...oom100
save = False

# 1. Imports and Set Up

In [2]:
from impt_functions import *
import csv
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
from fairlearn.metrics import *
from fairlearn.preprocessing import CorrelationRemover
from raiwidgets import FairnessDashboard
%matplotlib inline

In [3]:
# Dict used for saving overall results to a csv later
# Example: overall = {'Unmitigated':[], 'EG DP': [], 'EG EO': [],
#                   'EG EOO': [], 'EG FPRP': [], 'EG ERP': [],
#                   'GS DP': [], 'GS EO': [],
#                   'GS EOO': [], 'GS FPRP': [], 'GS ERP': []}
# Example: black_results = {'Unmitigated Black': [], 
#                          'EG DP Black': [], ..., 'GS ERP Black': []}

# Instantiate lists for holding results
pp_mitigated, pp_mitigated_black, pp_mitigated_white = [], [], []

# 2. Prepare data

In [4]:
data = get_data(data_file)

       score  repay_probability  race  repay_indices
0        601              75.21     1              1
1        693              95.15     1              1
2        791              98.62     1              1
3        637              86.69     1              1
4        775              98.45     1              1
...      ...                ...   ...            ...
99995    797              98.73     1              1
99996    562              58.57     1              1
99997    687              94.60     1              1
99998    589              70.61     1              1
99999    555              52.97     1              0

[100000 rows x 4 columns]


In [5]:
print(type(data))
print(data)

<class 'pandas.core.frame.DataFrame'>
       score  repay_probability  race  repay_indices
0        601              75.21     1              1
1        693              95.15     1              1
2        791              98.62     1              1
3        637              86.69     1              1
4        775              98.45     1              1
...      ...                ...   ...            ...
99995    797              98.73     1              1
99996    562              58.57     1              1
99997    687              94.60     1              1
99998    589              70.61     1              1
99999    555              52.97     1              0

[100000 rows x 4 columns]


In [ ]:
# Reference
# https://fairlearn.org/main/api_reference/fairlearn.preprocessing.html

# TODO: when running experiments, alter the alpha
alpha = 0.5  # between 0-1, alpha=1.0 filters out all information while for alpha=0.0 no mitigation applied

sensitive_feature_ids = 1  # list of columns to filter out this can be a sequence of either int ,in the case of numpy, or string, in the case of pandas
correl_remover = CorrelationRemover(sensitive_feature_ids, alpha) 

# TODO: figure out if it needs to be all the data or just training data?? check og paper
correl_remover.fit(data)
pp_data = correl_remover.transform(data)

In [6]:
X_train, X_test, y_train, y_test, race_train, race_test, sample_weight_train, sample_weight_test = prep_data(data=data, test_size=0.3, weight_index=1)

Here are the x values:  [[601   1]
 [693   1]
 [791   1]
 ...
 [687   1]
 [589   1]
 [555   1]] 

Here are the y values:  [1 1 1 ... 1 1 0]
Sample weights are all equal.


In [7]:
x = data[['score', 'race']].values
y = data['repay_indices'].values

# 3. Classifier

In [10]:
print('The classifier trained below is: ', model_name)

The classifier trained below is:  lgr


In [11]:
if model_name == 'dt':
    # Initialize classifier:
    classifier = DecisionTreeClassifier()
elif model_name == 'gnb':
    classifier = GaussianNB()
elif model_name == 'lgr':
    # Reference: https://towardsdatascience.com/logistic-regression-using-python-sklearn-numpy-mnist-handwriting-recognition-matplotlib-a6b31e2b166a
    classifier = LogisticRegression()
elif model_name == 'svm_linear':
    # Reference: https://www.datacamp.com/community/tutorials/svm-classification-scikit-learn-python
    classifier = svm.SVC(kernel='linear', probability=True)
elif model_name == 'gbt':
    # Reference: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html
    # Note: max_depth default is 3 but tune this parameter for best performance
    classifier = GradientBoostingClassifier(n_estimators=100) 
else:
    print('PROBLEM: input a specified classifier above')

## Train classifier and collect predictions
NOTE: atm sample_weight are all 1s

In [12]:
# Reference: https://www.datacamp.com/community/tutorials/decision-tree-classification-python

# Train the classifier:
model = classifier.fit(X_train,y_train, sample_weight_train)

# Make predictions with the classifier:
y_predict = model.predict(X_test)

# Scores on test set
test_scores = model.predict_proba(X_test)[:, 1]

### Evaluation of classifier overall

In [13]:
# Metrics
models_dict = {"pp_mitigated": (y_predict, test_scores)}
get_metrics_df(models_dict, y_test, race_test)

,Unmitigated
Overall selection rate,0.7368
Demographic parity difference,0.487291
Demographic parity ratio,0.38659
------,
Overall balanced error rate,0.154262
Balanced error rate difference,0.0161394
------,
True positive rate difference,0.232589
True negative rate difference,0.20031
False positive rate difference,0.20031


In [14]:
cm = confusion_matrix(y_test, y_predict)
print(cm)
results_dict = classification_report(y_test, y_predict, output_dict=True)
print(classification_report(y_test, y_predict))
# Add accuracy to the results list
pp_mitigated.append(round(results_dict['accuracy']*100, 2))
f1_micro, f1_weighted, f1_binary = get_f1_scores(y_test, y_predict)
f1_str = str(round(f1_micro*100, 2))+"/"+str(round(f1_weighted*100, 2))+"/"+str(round(f1_binary*100, 2))
# Add f1 scores to results list
pp_mitigated.append(f1_str)
# Add Selection rate to results list
sr = get_selection_rates(y_test, y_predict, race_test, 0)
unmitigated.append(round(sr*100, 2))
# Add Outcome rates to results list
tnr, tpr, fner, fper = evaluation_outcome_rates(y_test, y_predict, sample_weight_test)
pp_mitigated.append(round(tnr*100, 2))
pp_mitigated.append(round(tpr*100, 2))
pp_mitigated.append(round(fner*100, 2))
pp_mitigated.append(round(fper*100, 2))

[[ 6550  2130]
 [ 1346 19974]]
              precision    recall  f1-score   support

           0       0.83      0.75      0.79      8680
           1       0.90      0.94      0.92     21320

    accuracy                           0.88     30000
   macro avg       0.87      0.85      0.86     30000
weighted avg       0.88      0.88      0.88     30000

F1 score micro: 
0.8841333333333333
F1 score weighted: 
0.8824392024670211
F1 score binary: 
0.9199521002210759



The positional argument 'metric' has been replaced by a keyword argument 'metrics'. From version 0.10.0 passing it as a positional argument or as a keyword argument 'metric' will result in an error


Selection Rate Overall:  0.7368
TNR=TN/(TN+FP)=  0.7546082949308756
TPR=TP/(FP+FN)=  0.9368667917448406
FNER=FN/(FN+TP)=  0.06313320825515947
FPER=FP/(FP+TN)=  0.24539170506912442


### Cross-validated metrics

In [15]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, x, y, cv=5, scoring='f1_weighted')
print(scores)

[0.87973325 0.88484126 0.88081539 0.87985374 0.87599261]


### Delayed impact calculated

In [ ]:
di_black, di_white = calculate_delayed_impact(X_test, y_test, y_predict, race_test)
# Add DI to results list
di_str = str(round(di_black, 2))+"/"+str(round(di_white, 2))
pp_mitigated.append(di_str)

### Fairness Metric Evaluation of classifier

In [ ]:
dp_diff, eod_diff, eoo_dif, fpr_dif, er_dif = print_fairness_metrics(y_test, y_predict, race_test, sample_weight_test)

# Add the fairness metric differences to results list
pp_mitigated.append(round(dp_diff*100, 2))
pp_nmitigated.append(round(eod_diff*100, 2))
pp_mitigated.append(round(eoo_dif*100, 2))
pp_mitigated.append(round(fpr_dif*100, 2))
pp_mitigated.append(round(er_dif*100, 2))
pp_mitigated.append(alpha)

### Evaluation of classifier by race

In [ ]:
results_black, results_white = evaluation_by_race(X_test, y_test, race_test, y_predict, sample_weight_test)
pp_mitigated_black.extend(results_black)
pp_mitigated_black.append(alpha)
pp_mitigated_white.extend(results_white)
pp_mitigated_white.append(alpha)

### Save results to dictionaries

In [ ]:
run_key = 'Pre-proc Mitigated'
overall_results_dict = add_values_in_dict({}, run_key, pp_mitigated)
black_results_dict = add_values_in_dict({}, run_key, pp_mitigated_black)
white_results_dict = add_values_in_dict({}, run_key, pp_mitigated_white)
print(overall_results_dict)
print(black_results_dict)
print(white_results_dict)

# 4. Save results to csv files

In [ ]:
# I'll want to compare the preproc mitigated to the unmitigated results from before
# TODO: add alpha as a dict key!!

In [ ]:
# To use below!!
if save == True:
    overall_fieldnames = ['Run', 'Acc', 'F1micro/F1w/F1bsr', 'SelectionRate', 'TNR rate', 'TPR rate', 'FNER', 'FPER', 'DIB/DIW', 'DP Diff', 'EO Diff', 'TPR Diff', 'FPR Diff', 'ER Diff', 'Alpha']
    byrace_fieldnames = ['Run', 'Acc', 'F1micro/F1w/F1bsr', 'SelectionRate', 'TNR rate', 'TPR rate', 'FNER', 'FPER', 'DI', 'Alpha']
    save_dict_2_csv(overall_results_dict, overall_fieldnames, model_name+'_overall_results.csv')
    save_dict_2_csv(black_results_dict, byrace_fieldnames, model_name+'_black_results.csv')
    save_dict_2_csv(white_results_dict, byrace_fieldnames, model_name+'_white_results.csv')